# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** K

**Names:**

*  Xavier Jeanmonod
*  Adrian Baudat
*  Simon Wicky

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.2 Network sampling

#### Exercise 2.7: Random walk on the Facebook network

In [7]:
import requests
import random

# Base url of the API
URL_TEMPLATE = 'http://iccluster040.iccluster.epfl.ch:5050/v1.0/facebook?user={user_id}'

def get_data(user_id):
    # The actual url to call 
    url = URL_TEMPLATE.format(user_id=user_id)
    # Execute the HTTP Get request
    response = requests.get(url)
    # Format the json response as a Python dict
    return response.json()

def random_walk(source, number):
    total_age = 0
    node = source
    for i in range(number):
        node_data = get_data(node)
        total_age += node_data['age']
        neighbors = node_data['friends']
        node = random.choice(neighbors)
    return total_age / number

#Number of nodes to visit
NUMBER_NODES = 300

#Node to start the random walk from
starting_node = 'f30ff3966f16ed62f5165a229a19b319'

print("Average age with 10 nodes (first run):", random_walk(starting_node, 10))
print("Average age with 10 nodes (second run):", random_walk(starting_node, 10))
print("Average age with 10 nodes (third run):", random_walk(starting_node, 10))

print("Average age with 50 nodes (first run):", random_walk(starting_node, 50))
print("Average age with 50 nodes (second run):", random_walk(starting_node, 50))
print("Average age with 50 nodes (third run):", random_walk(starting_node, 50))

print("Average age with 500 nodes (first run):", random_walk(starting_node, 500))
print("Average age with 500 nodes (second run):", random_walk(starting_node, 500))
print("Average age with 500 nodes (third run):", random_walk(starting_node, 500))

Average age with 10 nodes (first run): 24.0
Average age with 10 nodes (second run): 29.5
Average age with 10 nodes (third run): 14.6
Average age with 50 nodes (first run): 18.38
Average age with 50 nodes (second run): 20.62
Average age with 50 nodes (third run): 22.58
Average age with 500 nodes (first run): 24.882
Average age with 500 nodes (second run): 24.952
Average age with 500 nodes (third run): 22.186


As we can see, the average varies quite a bit with the number of nodes we visit, and it also changes between different runs. However, with 500 nodes the average starts to stabilize around 22 years old.

#### Exercise 2.8

Our estimate (22 years old) is very far from the actual average age which is 43.3 years old.
The reason for this may come from the fact users with many friends are counted more than ones with fewer friends. If younger people have on average more friends on Facebook than older people, this would introduce bias towards younger ages. To fix this, we use an unbiased estimator.

In [10]:
def unbiased_random_walk(source, number):
    data = []
    node = source
    for i in range(number):
        node_data = get_data(node)
        neighbors = node_data['friends']
        data.append((node_data['age'], len(neighbors)))
        node = random.choice(neighbors)
    num = 0
    denom = 0
    for age,degree in data:
        num += age/degree
        denom += 1/degree
    return num/denom

print("Average age with 10 nodes (first run):", unbiased_random_walk(starting_node, 10))
print("Average age with 10 nodes (second run):", unbiased_random_walk(starting_node, 10))
print("Average age with 10 nodes (third run):", unbiased_random_walk(starting_node, 10))

print("Average age with 50 nodes (first run):", unbiased_random_walk(starting_node, 50))
print("Average age with 50 nodes (second run):", unbiased_random_walk(starting_node, 50))
print("Average age with 50 nodes (third run):", unbiased_random_walk(starting_node, 50))

print("Average age with 500 nodes (first run):", unbiased_random_walk(starting_node, 500))
print("Average age with 500 nodes (second run):", unbiased_random_walk(starting_node, 500))
print("Average age with 500 nodes (third run):", unbiased_random_walk(starting_node, 500))

Average age with 10 nodes (first run): 36.083506486213295
Average age with 10 nodes (second run): 41.305882352941175
Average age with 10 nodes (third run): 16.30852713178295
Average age with 50 nodes (first run): 22.750955266283782
Average age with 50 nodes (second run): 47.569196146297344
Average age with 50 nodes (third run): 43.35284911905964
Average age with 500 nodes (first run): 41.676484217025795
Average age with 500 nodes (second run): 41.477240435597665
Average age with 500 nodes (third run): 43.32715123676788


As we can see, using an unbiased estimator yields a much better approximation of the average age, which validates our hypothesis.